In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score, recall_score

In [2]:
## 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# jupyter展示图片，非内嵌显示
# tk: 显示出来，inline：内嵌显示，默认为inline
%matplotlib tk

In [4]:
# 1. 文件数据读取
df = pd.read_csv('../data/result_process02', sep=',')
# 如果有某值为nan，进行删除操作
df.dropna(axis=0, how='any', inplace=True)
df.head(10)

,label,has_date,jieba_cut_content,content_sema
0,1.0,1,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,8.456151
1,0.0,1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,7.486084
2,1.0,1,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...,7.175171
3,1.0,1,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...,7.565682
4,1.0,1,这是 一封 HTML 格式 信件 ！ - - - - - - - ...,2.063409
5,1.0,1,TO ： 贵 公司 经理 、 财务 您好 ！ 深圳市 春洋 贸易 有...,7.143747
6,0.0,1,那 他 为什么 不 愿意 起诉 ， 既然 这样 了 ！ 起诉 后 ...,4.807568
7,1.0,1,尊敬 的 负责人 （ 经理 ／ 财务 ） ： 您好 ！ 我 是 深...,6.593684
8,1.0,1,您好 以下 是 特别 为 阁下 发 的 香港 信息 ( 图片 ...,7.611074
9,0.0,1,我 觉得 ， 负债 不要紧 ， 最 重要 的 是 能 负得起 这个 ...,7.041340


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64284 entries, 0 to 64619
Data columns (total 4 columns):
label                64284 non-null float64
has_date             64284 non-null int64
jieba_cut_content    64284 non-null object
content_sema         64284 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.5+ MB


In [6]:
# 2. 数据分割
x_train, x_test, y_train, y_test = train_test_split(df[['has_date','jieba_cut_content','content_sema']], df['label'], test_size=0.2, random_state=0)
print("训练数据集大小:%d" % x_train.shape[0])
print("测试数据集大小:%d" % x_test.shape[0])

训练数据集大小:51427
测试数据集大小:12857


In [7]:
x_train.head(5)

,has_date,jieba_cut_content,content_sema
18810,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.849239
61632,1,声音 变换器 ： 适用 于 不同 型号 的 手机 ， 电话 ， 网络...,7.242064
5822,1,http : / / tengyingge . blogchina . co...,8.289843
5881,1,刚 开始 ， 是 会 有点 不 习惯 的 。 慢慢来 。 ps ：...,7.063309
26338,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.700159


In [8]:
# 3. 开始模型训练
# 3.1 特征工程，将文本数据转换为数值型数据
vectorizer = CountVectorizer()
transfromer = TfidfTransformer(norm='l2', use_idf=True)
svd = TruncatedSVD(n_components=20)
jieba_cut_content = list(x_train['jieba_cut_content'].astype('str'))
vectorizer_model = vectorizer.fit(jieba_cut_content)
df0 = vectorizer_model.transform(jieba_cut_content)
transfromer_model = transfromer.fit(df0)
df1 = transfromer_model.transform(df0)
svd_model = svd.fit(df1)
df2 = svd_model.transform(df1)
data = pd.DataFrame(df2)
print(data.head(5))
print(data.info())

         0         1         2         3         4         5         6   \
0  0.340289  0.220012 -0.237467 -0.039931 -0.070145  0.002502 -0.160798   
1  0.014968  0.064740  0.027003 -0.004678  0.008171  0.007474  0.016318   
2  0.011862  0.072033  0.047002  0.001412 -0.002085  0.035913  0.009240   
3  0.019714  0.117439  0.132451  0.060417 -0.103507 -0.074860 -0.012258   
4  0.346257  0.215336 -0.234020 -0.043112 -0.071055 -0.000038 -0.162626   

         7         8         9         10        11        12        13  \
0 -0.009780  0.201847 -0.010390 -0.199888  0.039180 -0.065096  0.429644   
1  0.046212  0.029362 -0.000188  0.007847  0.023392  0.041790 -0.012305   
2 -0.012144 -0.016673  0.000278  0.001125 -0.004093 -0.002294 -0.003597   
3 -0.005661 -0.027397  0.000312 -0.016888 -0.013009 -0.003257  0.017835   
4 -0.009081  0.199177 -0.010192 -0.195555  0.037443 -0.061499  0.417632   

         14        15        16        17        18        19  
0 -0.478478  0.207323  0.098653 -0

In [9]:
# 3.2 数据合并
data['has_date'] = list(x_train['has_date'])
data['content_sema'] = list(x_train['content_sema'])
print(data.head(5))
print(data.info())

          0         1         2         3         4         5         6  \
0  0.340289  0.220012 -0.237467 -0.039931 -0.070145  0.002502 -0.160798   
1  0.014968  0.064740  0.027003 -0.004678  0.008171  0.007474  0.016318   
2  0.011862  0.072033  0.047002  0.001412 -0.002085  0.035913  0.009240   
3  0.019714  0.117439  0.132451  0.060417 -0.103507 -0.074860 -0.012258   
4  0.346257  0.215336 -0.234020 -0.043112 -0.071055 -0.000038 -0.162626   

          7         8         9      ...             12        13        14  \
0 -0.009780  0.201847 -0.010390      ...      -0.065096  0.429644 -0.478478   
1  0.046212  0.029362 -0.000188      ...       0.041790 -0.012305  0.013032   
2 -0.012144 -0.016673  0.000278      ...      -0.002294 -0.003597 -0.002394   
3 -0.005661 -0.027397  0.000312      ...      -0.003257  0.017835  0.001864   
4 -0.009081  0.199177 -0.010192      ...      -0.061499  0.417632 -0.468417   

         15        16        17        18        19  has_date  content_sem

In [10]:
%%time
tree = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=0)
model = tree.fit(data, y_train)

Wall time: 635 ms


In [11]:
# 4.1 对测试数据进行特征转换
jieba_cut_content_test = list(x_test['jieba_cut_content'].astype('str'))
data_test = pd.DataFrame(svd_model.transform(transfromer_model.transform(vectorizer_model.transform(jieba_cut_content_test))))
data_test['has_date'] = list(x_test['has_date'])
data_test['content_sema'] = list(x_test['content_sema'])
print(data_test.head(5))
print(data_test.info())

          0         1         2         3         4         5         6  \
0  0.537283  0.009050 -0.087588  0.028956  0.000889 -0.032738 -0.157590   
1  0.210902  0.131679 -0.090075 -0.086949 -0.033303  0.039749  0.007000   
2  0.045151  0.277285  0.233893  0.051026 -0.074039 -0.016267  0.010703   
3  0.032907  0.163067  0.113488  0.002453  0.053725  0.008273  0.004268   
4  0.035278  0.065206  0.026071 -0.023764  0.072928 -0.005787 -0.003118   

          7         8         9      ...             12        13        14  \
0  0.009190 -0.134565  0.009340      ...       0.079884  0.005519 -0.016735   
1 -0.008299  0.011677 -0.000316      ...      -0.025513  0.082346 -0.057004   
2 -0.014371  0.100288 -0.002601      ...      -0.067399 -0.039166 -0.031012   
3 -0.020094  0.117820 -0.002484      ...      -0.062200 -0.059201 -0.012805   
4 -0.008325  0.008179  0.000842      ...      -0.001836 -0.006221 -0.003272   

         15        16        17        18        19  has_date  content_sem

In [12]:
# 4.2 对测试数据进行预测
y_predict = model.predict(data_test)

In [13]:
# 5. 效果评估
print("准确率为:%.5f" % precision_score(y_test, y_predict))
print("召回率为:%.5f" % recall_score(y_test, y_predict))
print("F1值为:%.5f" % f1_score(y_test, y_predict))

准确率为:0.96911
召回率为:0.98182
F1值为:0.97542
